<a href="https://colab.research.google.com/github/SibiSenthur/Python-Projects-/blob/master/Sibi_Senthur_Pandas_Repo_Utility_Fucntions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
# Importing the necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
from scipy import stats
from scipy.stats import norm, skew #for some statistics
import datetime as dt
import calendar

In [43]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [44]:
df_ccinfo = pd.read_csv('/content/drive/My Drive/Colab Notebooks/cc_info.csv', sep = ',')
df_transactions = pd.read_csv('/content/drive/My Drive/Colab Notebooks/transactions.csv', sep = ',')

In [45]:
df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train.csv', sep = ',')

In [47]:
def my_agg(x):
    names = {
        'age_mean': x['Age'].mean(),
        'age_max':  x['Age'].max(),
        'fare_mean': x['Fare'].mean(),
        'fare_max': x['Fare'].max()
    } 

    return pd.Series(names, index=[ key for key in names.keys()]) # all the columns you create in the previous dictionary will be in this list comprehension

df_train.groupby('Pclass').apply(my_agg)

,age_mean,age_max,fare_mean,fare_max
Pclass,,,,
1,38.233441,80.0,84.154687,512.3292
2,29.877630,70.0,20.662183,73.5000
3,25.140620,74.0,13.675550,69.5500


In [48]:
# Do some fast feature eng on the DF
d = {"gender":["male", "female", "male"], "color":["red", "green", "blue"], "age":[25, 30, 15]}
df = pd.DataFrame(d)
# Solution
map_dict = {"male":"M", "female":"F"}
df["gender_mapped"] = df["gender"].map(map_dict) # using dictionaries to map values
df["color_factorized"] = df["color"].factorize()[0] # using factorize: returns a tuple of arrays (array([0, 1, 2]), Index(['red', 'green', 'blue'], dtype='object')) that's why we select [0]
df["age_compared_boolean"] = df["age"] < 18 # return a True False boolean value

In [49]:
df

,gender,color,age,gender_mapped,color_factorized,age_compared_boolean
0,male,red,25,M,0,False
1,female,green,30,F,1,False
2,male,blue,15,M,2,True


In [41]:
d = {"customer":["A", "B", "C", "D", "E"], "sales":[100, "100", 50, 550.20, "375.25"]}
df = pd.DataFrame(d)

df["sales"].apply(type) # Wow we can see that we have int, str, floats inn one column
df["sales"].apply(type).value_counts() # See the number of each value

<class 'str'>      2
<class 'int'>      2
<class 'float'>    1
Name: sales, dtype: int64

In [24]:
# check Trick 91 for an example
def generate_sample_data_datetime(): # creates a fake df for testing
    number_or_rows = 365*24
    num_cols = 2
    cols = ["sales", "customers"]
    df = pd.DataFrame(np.random.randint(1, 20, size = (number_or_rows, num_cols)), columns=cols)
    df.index = pd.util.testing.makeDateIndex(number_or_rows, freq="H")
    return df

In [25]:
df = generate_sample_data_datetime()

In [30]:
# Step 1: resample by D. Basically agregate by day and use to_frame() to convert it to frame
daily_sales = df.resample("D")["sales"].sum().to_frame()
daily_sales
# Step 2: filter weekends
weekends_sales = daily_sales[daily_sales.index.dayofweek.isin([5, 6])]

In [31]:
print(weekends_sales)

            sales
2000-01-01    314
2000-01-02    247
2000-01-08    255
2000-01-09    242
2000-01-15    242
...           ...
2000-12-16    248
2000-12-17    251
2000-12-23    246
2000-12-24    214
2000-12-30    190

[105 rows x 1 columns]


In [32]:
df.head(10)

,sales,customers
2000-01-01 00:00:00,8,10
2000-01-01 01:00:00,16,9
2000-01-01 02:00:00,7,4
2000-01-01 03:00:00,10,17
2000-01-01 04:00:00,15,17
2000-01-01 05:00:00,10,3
2000-01-01 06:00:00,13,14
2000-01-01 07:00:00,18,9
2000-01-01 08:00:00,18,2
2000-01-01 09:00:00,5,9


In [ ]:
print("Shape of credit card info",df_ccinfo.shape)
print("shape of transactions info", df_transactions.shape)

Shape of credit card info (984, 5)
shape of transactions info (294588, 5)


In [ ]:
df_transactions.head(10)

,credit_card,date,transaction_dollar_amount,Long,Lat
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719
5,1003715054175576,2015-10-17 21:28:57,121.60,-80.243565,40.260887
6,1003715054175576,2015-08-29 18:34:04,122.65,-80.238186,40.245928
7,1003715054175576,2015-08-14 21:34:39,98.00,-80.244063,40.249143
8,1003715054175576,2015-09-17 19:20:37,95.20,-2.833179,34.745015
9,1003715054175576,2015-09-11 18:59:04,57.95,-80.287528,40.267874


In [ ]:
df_transactions['date'] = pd.to_datetime(df_transactions['date'])

In [ ]:
df_ccinfo.head(5)

,credit_card,city,state,zipcode,credit_card_limit
0,1280981422329509,Dallas,PA,18612,6000
1,9737219864179988,Houston,PA,15342,16000
2,4749889059323202,Auburn,MA,1501,14000
3,9591503562024072,Orlando,WV,26412,18000
4,2095640259001271,New York,NY,10001,20000


In [ ]:
df_combine = pd.merge(df_transactions, df_ccinfo, how='left') #merge
df_combine['Month'] = df_combine['date'].dt.month #month
df_combine['Day_of_Week'] = df_combine['date'].dt.day_name() #day_name

In [ ]:
df_combine.dtypes

credit_card                           int64
date                         datetime64[ns]
transaction_dollar_amount           float64
Long                                float64
Lat                                 float64
city                                 object
state                                object
zipcode                               int64
credit_card_limit                     int64
Month                                 int64
Day_of_Week                          object
dtype: object

In [ ]:
df_combine['Month'] = df_combine['Month'].apply(lambda x: calendar.month_abbr[x]) #obtaining the month name;

In [ ]:
df_combine['Month'].value_counts()

Aug    99167
Sep    96026
Oct    95014
Jul     4381
Name: Month, dtype: int64

In [ ]:
df_combine.head(5)

,credit_card,date,transaction_dollar_amount,Long,Lat,city,state,zipcode,credit_card_limit,Month,Day_of_Week
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370,Houston,PA,15342,20000,Sep,Friday
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114,Houston,PA,15342,20000,Oct,Saturday
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004,Houston,PA,15342,20000,Oct,Monday
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895,Houston,PA,15342,20000,Oct,Thursday
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719,Houston,PA,15342,20000,Oct,Monday


In [ ]:
df_combine['new_date'] = [d.date() for d in df_combine['date']]
df_combine['new_time'] = [d.time() for d in df_combine['date']]

In [ ]:
df_combine.head(5)

,credit_card,date,transaction_dollar_amount,Long,Lat,city,state,zipcode,credit_card_limit,Month,Day_of_Week,new_date,new_time
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370,Houston,PA,15342,20000,Sep,Friday,2015-09-11,00:32:40
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114,Houston,PA,15342,20000,Oct,Saturday,2015-10-24,22:23:08
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004,Houston,PA,15342,20000,Oct,Monday,2015-10-26,18:19:36
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895,Houston,PA,15342,20000,Oct,Thursday,2015-10-22,19:41:10
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719,Houston,PA,15342,20000,Oct,Monday,2015-10-26,20:08:22


In [ ]:
df_combine['new_date'] = pd.to_datetime(df_combine['new_date'])

In [ ]:
toy = pd.DataFrame({
'id': [1,2,3,
       1,2,3,
       1,2,3],
'date': ['2015-05-13', '2015-05-13', '2015-05-13', 
         '2016-02-12', '2016-02-12', '2016-02-12', 
         '2018-07-23', '2018-07-23', '2018-07-23'],
'my_metric': [395, 634, 165, 
              144, 305, 293, 
              23, 395, 242]
})

In [ ]:
toy.head(10)

,id,date,my_metric
0,1,2015-05-13,395
1,2,2015-05-13,634
2,3,2015-05-13,165
3,1,2016-02-12,144
4,2,2016-02-12,305
5,3,2016-02-12,293
6,1,2018-07-23,23
7,2,2018-07-23,395
8,3,2018-07-23,242


In [ ]:
#https://stackoverflow.com/questions/53609003/moving-average-on-pandas-groupby-object-that-respects-time
def rolling_average(x, dt):
      xt = x.sort_index().groupby(lambda x: x.time()).rolling(window=dt).mean()
      xt.index = xt.index.droplevel(0)
      return xt
dt='3D'
# Group by the 'credit_card' column
g = df_combine.groupby('credit_card')
df = g.apply(rolling_average, dt=dt)
# Massage the data to appropriate format
df.index = df.index.droplevel(0)
df = df.reset_index().drop_duplicates(keep='last', subset=['credit_card', 'date'])

AttributeError: ignored